# Blank Slate UBI chart generation

This notebook creates the charts for the report.

In [1]:
import pandas as pd
import json
from policyengine_core.charts import format_fig
import plotly.express as px

df = pd.read_csv('ubi_amounts.csv')

with open("json_data.json", "r") as file:
    json_data = json.load(file)
    

# Melt the DataFrame to make it suitable for Plotly
melted_df = df.sort_values("flat_tax_rate").melt(id_vars=['flat_tax_rate'], 
                    value_vars=['ubi_0_5', 'ubi_6_17', 'ubi_18_64', 'ubi_65_plus', 'ubi_disability'],
                    var_name='UBI Category', value_name='Amount')

melted_df['UBI Category'] = melted_df['UBI Category'].map({
    "ubi_0_5": "0-5",
    "ubi_6_17": "6-17",
    "ubi_18_64": "18-64",
    "ubi_65_plus": "65+",
    "ubi_disability": "Disability"
})

# Create the plot
fig = px.line(melted_df, x='flat_tax_rate', y='Amount', color='UBI Category',
              labels={'flat_tax_rate': 'Flat Tax Rate', 'Amount': 'UBI Amount'},
              title='UBI Amounts by Flat Tax Rate', color_discrete_sequence=px.colors.qualitative.D3)

# Show the plot
format_fig(fig).update_layout(
    legend_title="UBI category",
    yaxis_tickformat = "$,.0f",
    yaxis_title="Yearly amount",
    xaxis_title="Flat tax rate",
    xaxis_tickformat=".0%",
    title="UBI amounts by flat tax rate"
)

In [2]:
import pandas as pd

def create_dataframes_from_json(json_data, df):
    dataframes = {}
    for api_id, metrics in json_data.items():
        for metric_name, metric_data in metrics.items():
            if metric_name not in dataframes:
                dataframes[metric_name] = pd.DataFrame()

            # Flatten the metric data and add API ID
            flat_data = pd.DataFrame([{**{"rounded_ubi_api_id": api_id}, **metric_data}])
            dataframes[metric_name] = pd.concat([dataframes[metric_name], flat_data], ignore_index=True)

    # Ensure data type consistency for merging
    df['rounded_ubi_api_id'] = df['rounded_ubi_api_id'].astype(str)

    # Merge each dataframe with the df dataframe to add the flat_tax_rate
    for metric_name, metric_df in dataframes.items():
        metric_df['rounded_ubi_api_id'] = metric_df['rounded_ubi_api_id'].astype(str)
        merged_df = metric_df.merge(df[['rounded_ubi_api_id', 'flat_tax_rate']], on='rounded_ubi_api_id', how='left')
        # Sort the DataFrame by flat_tax_rate
        dataframes[metric_name] = merged_df.sort_values(by='flat_tax_rate')

    return dataframes

# Assuming 'json_data' is your JSON data and 'df' is your existing DataFrame
dfs = create_dataframes_from_json(json_data, df)

# Accessing and printing a specific dataframe, for example, 'budget'
budget_df = dfs.get('budget', None)

import pandas as pd
import plotly.express as px

def calculate_percentage_change(dataframe, column_name):
    data = dataframe[column_name].apply(pd.Series)
    data['% change'] = ((data['reform'] - data['baseline']) / data['baseline'])
    data['flat_tax_rate'] = dataframe['flat_tax_rate']
    return data[['flat_tax_rate', '% change']].rename(columns={'% change': f'% change {column_name}'})

# Assuming dfs['inequality'] is your DataFrame
gini_change = calculate_percentage_change(dfs['inequality'], 'gini')
top_10_pct_share_change = calculate_percentage_change(dfs['inequality'], 'top_10_pct_share')
top_1_pct_share_change = calculate_percentage_change(dfs['inequality'], 'top_1_pct_share')

labour_supply_response = dfs["labour_supply_response"][["total_change", "flat_tax_rate"]].rename(columns=dict(total_change="labor_supply"))
labour_supply_response.labor_supply /= 10320e9
# Combine the data
combined_data = gini_change
combined_data = combined_data.merge(top_10_pct_share_change, on='flat_tax_rate')
combined_data = combined_data.merge(top_1_pct_share_change, on='flat_tax_rate')
combined_data = combined_data.merge(labour_supply_response, on='flat_tax_rate')

combined_data.rename(columns={
    "flat_tax_rate": "Flat tax rate",
    "labor_supply": "Labor supply",
    "% change gini": "Gini index",
    "% change top_10_pct_share": "Top 10% share",
    "% change top_1_pct_share": "Top 1% share",
}, inplace=True)

fig = px.line(combined_data, x='Flat tax rate', y=['Gini index', 'Top 10% share', 'Top 1% share'],
              labels={'value': '% Change', 'variable': 'Metric'},
              title='Change in inequality metrics by flat tax rate',
              color_discrete_sequence=px.colors.qualitative.D3)
fig = format_fig(fig).update_layout(
    yaxis_tickformat = "+.0%",
    yaxis_title="Percentage change",
    xaxis_title="Flat tax rate",
    xaxis_tickformat=".0%",
    title="Change in inequality metrics by flat tax rate"
)

# Add dashed line at y=0

from policyengine_core.charts import *

fig.add_shape(
    type="line",
    x0=0,
    y0=0,
    x1=1,
    y1=0,
    line=dict(
        color=DARK_GRAY,
        width=1,
        dash="dash",
    ),
    xref="paper",
    yref="y",
)


In [3]:
def calculate_percentage_change_poverty(dataframe, column_name):
    data = dataframe[column_name].apply(pd.Series)
    for category in ['adult', 'all', 'child', 'senior']:
        data[f'% change {category}'] = ((data[category].apply(lambda x: x['reform']) - 
                                         data[category].apply(lambda x: x['baseline'])) / 
                                         data[category].apply(lambda x: x['baseline']))
    data['flat_tax_rate'] = dataframe['flat_tax_rate']
    return data[['flat_tax_rate', f'% change adult', f'% change all', f'% change child', f'% change senior']]

# Assuming dfs['poverty'] is your DataFrame
poverty_change = calculate_percentage_change_poverty(dfs['poverty'], 'poverty')

poverty_change.rename(columns={
    "flat_tax_rate": "Flat tax rate",
    "% change all": "Poverty",
    "% change adult": "Adult poverty",
    "% change child": "Child poverty",
    "% change senior": "Senior poverty",
}, inplace=True)


fig = px.line(poverty_change, x='Flat tax rate', y=['Poverty', 'Adult poverty', 'Child poverty', 'Senior poverty'],
              labels={'value': '% Change', 'variable': 'Category'},
              title='Change in poverty metrics by flat tax rate',
                color_discrete_sequence=px.colors.qualitative.D3)
fig = format_fig(fig).update_layout(
    yaxis_tickformat = "+.0%",
    yaxis_title="Percentage change",
    xaxis_title="Flat tax rate",
    xaxis_tickformat=".0%",
    title="Change in poverty metrics by flat tax rate"
)

# Add dashed line at y=0

fig.add_shape(
    type="line",
    x0=0,
    y0=0,
    x1=1,
    y1=0,
    line=dict(
        color=DARK_GRAY,
        width=1,
        dash="dash",
    ),
    xref="paper",
    yref="y",
)


In [4]:
deep_poverty_change = calculate_percentage_change_poverty(dfs['poverty'], 'deep_poverty')


deep_poverty_change.rename(columns={
    "flat_tax_rate": "Flat tax rate",
    "% change all": "Deep poverty",
    "% change adult": "Adult deep poverty",
    "% change child": "Child deep poverty",
    "% change senior": "Senior deep poverty",
}, inplace=True)

fig = px.line(deep_poverty_change, x='Flat tax rate', y=['Deep poverty', 'Adult deep poverty', 'Child deep poverty', 'Senior deep poverty'],
                labels={'value': '% Change', 'variable': 'Category'},
                title='Change in deep poverty metrics by flat tax rate',
                    color_discrete_sequence=px.colors.qualitative.D3)

fig = format_fig(fig).update_layout(
    yaxis_tickformat = "+.0%",
    yaxis_title="Percentage change",
    xaxis_title="Flat tax rate",
    xaxis_tickformat=".0%",
    title="Change in deep poverty metrics by flat tax rate"
)
fig.add_shape(
    type="line",
    x0=0,
    y0=0,
    x1=1,
    y1=0,
    line=dict(
        color=DARK_GRAY,
        width=1,
        dash="dash",
    ),
    xref="paper",
    yref="y",
)


In [5]:
import pandas as pd
import plotly.express as px

# Define a function to determine increase (1) or decrease (-1)
def increase_or_decrease(value):
    return 1 if value > 0 else -1

labor_supply_change = combined_data[["Flat tax rate", "Labor supply"]]

gini_change.columns = ["Flat tax rate", "Gini index"]
top_10_pct_share_change.columns = ["Flat tax rate", "Top 10% share"]
top_1_pct_share_change.columns = ["Flat tax rate", "Top 1% share"]

# Combine the percentage change data from poverty, deep poverty, and inequality
# Assuming dfs['poverty'] and other DataFrames are already defined
combined_change = dfs['poverty'][['flat_tax_rate']].copy().rename(columns={'flat_tax_rate': 'Flat tax rate'})
combined_change_categorical = combined_change.copy()

# Assume you have other dataframes like gini_change, top_10_pct_share_change, etc.
for dataframe in [labor_supply_change, gini_change, top_10_pct_share_change, top_1_pct_share_change, poverty_change]:
    dataframe.sort_values('Flat tax rate', inplace=True)
    for col in dataframe.columns:
        if col != 'Flat tax rate':
            if col == "Labor supply":
                combined_change_categorical[col] = dataframe[col].apply(lambda x: -1 if x > 0 else 1)
            else:
                combined_change_categorical[col] = dataframe[col].apply(increase_or_decrease)
            combined_change[col] = dataframe[col]

import pandas as pd
import plotly.figure_factory as ff

# Assume combined_change is a DataFrame with the given structure
# Mapping the percentage change to colors
combined_change_mapped = combined_change_categorical.copy()

import pandas as pd
import plotly.figure_factory as ff
import numpy as np

# Rest of your code where you define combined_change and combined_change_mapped...

# Preparing the data for the heatmap
heatmap_data = combined_change_mapped.set_index('Flat tax rate').T

# The x and y coordinates for the heatmap should be lists or arrays
x_values = combined_change_mapped['Flat tax rate'].tolist()
y_values = heatmap_data.index.tolist()

# The z values and annotations must be 2D arrays that match the length of x and y
#heatmap_data = heatmap_data.replace({'Increase': "+", 'Decrease': "-"})
z_values = heatmap_data.values.T  # Transposing to match the dimensions
annotations = combined_change.set_index('Flat tax rate').T.values.T  # This should be the same shape as z_values

heatmap_annotations = pd.DataFrame(annotations, columns=y_values, index=x_values)
heatmap_annotations = heatmap_annotations.apply(lambda x: x.apply(lambda y: f"{y:+.0%}"))
# Assume combined_change is a DataFrame with the given structure
# Mapping the percentage change to colors

from policyengine_core.charts import *
y_values = [
    "Labor supply",
    "Gini index",
    "Top 10% income share",
    "Top 1% income share",
    "Adult poverty rate",
    "Poverty rate",
    "Child poverty rate",
    "Senior poverty rate",
]

# Creating the heatmap using Figure Factory to properly display categorical data
fig = ff.create_annotated_heatmap(
    z=z_values.T,  # Transposing to match the dimensions
    x=x_values,
    y=y_values,
    annotation_text=heatmap_annotations.values.T,
    colorscale=[BLUE, DARK_GRAY],
    showscale=False
)

# Update the layout to remove the color axis scale (legend)
format_fig(fig).update_layout(
    title='Change in metric under optimal UBI by flat tax rate',
    xaxis=dict(title='Flat tax rate', side='top', tickmode='array', tickvals=x_values, ticktext=[f"{x:.0%}" for x in x_values]),
    yaxis=dict(title='Metric', tickmode='array', tickvals=list(range(len(y_values))), ticktext=y_values),
    margin=dict(l=150, t=150),  # Adjust margins to fit y-axis labels
    height=600,
    width=1200
)

# Show the figure
fig.show()


C:\Users\berko\AppData\Local\Temp\ipykernel_29464\2574770632.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

